<a href="https://colab.research.google.com/github/AnDrIi6/Text-Normalization-Challenge---English-Language-2-/blob/main/English_Language_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import gc
import xgboost as xgb
import re
from google.colab import files
from sklearn.model_selection import train_test_split

# Завантаження файлу вручну
uploaded = files.upload()  # Відкриє вікно для вибору файлу

# Отримуємо ім'я завантаженого файлу
file_name = list(uploaded.keys())[0]
print(f"Завантажений файл: {file_name}")

# Перевіряємо формат файлу та завантажуємо дані
if file_name.endswith('.csv'):
    df = pd.read_csv(file_name)
elif file_name.endswith(('.xls', '.xlsx')):
    df = pd.read_excel(file_name)
else:
    raise ValueError("Файл має бути у форматі .csv, .xls або .xlsx")

# Налаштування параметрів
max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0
max_data_size = 2000000

x_data = []
y_data = pd.factorize(df['class'])
labels = y_data[1]
y_data = y_data[0]
gc.collect()

# Перетворення символів у числове представлення
for x in df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_data.append(x_row)

# Контекстне вікно
def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    pre = [pre for x in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i : i + pad_size * 2 + 1]:
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data

# Обмеження розміру даних
x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]
x_data = np.array(context_window_transform(x_data, pad_size))
gc.collect()
x_data = np.array(x_data)
y_data = np.array(y_data)

# Інформація про дані
print('Total number of samples:', len(x_data))
print('Use: ', max_data_size)

print('x_data sample:')
print(x_data[0])
print('y_data sample:')
print(y_data[0])
print('labels:')
print(labels)


Saving en_train4_converted.xlsx to en_train4_converted.xlsx
Завантажений файл: en_train4_converted.xlsx
Total number of samples: 99742
Use:  2000000
x_data sample:
[ -1   0   0   0   0   0   0   0   0   0   0  -1  66 114 105 108 108  97
 110 116  97 105  -1 105 115   0   0   0   0   0   0   0   0  -1]
y_data sample:
0
labels:
Index(['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM', 'DECIMAL',
       'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC', 'DIGIT',
       'FRACTION', 'TELEPHONE', 'ADDRESS'],
      dtype='object')


In [ ]:
x_train = x_data
y_train = y_data
gc.collect()

x_train, x_valid, y_train, y_valid= train_test_split(x_train, y_train,
                                                      test_size=0.1, random_state=2017)
gc.collect()
num_class = len(labels)
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(dvalid, 'valid'), (dtrain, 'train')]

param = {'objective':'multi:softmax',
         'eta':'0.3', 'max_depth':10,
         'silent':1, 'nthread':-1,
         'num_class':num_class,
         'eval_metric':'merror'}
model = xgb.train(param, dtrain, 50, watchlist, early_stopping_rounds=20,
                  verbose_eval=10)
gc.collect()

pred = model.predict(dvalid)
pred = [labels[int(x)] for x in pred]
y_valid = [labels[x] for x in y_valid]
x_valid = [ [ chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]] for y in x_valid]
x_valid = [''.join(x) for x in x_valid]
x_valid = [re.sub('a+$', '', x) for x in x_valid]

gc.collect()

df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = x_valid
df_pred['predict'] = pred
df_pred['target'] = y_valid
df_pred.to_csv(os.path.join(out_path, 'pred.csv'))

df_erros = df_pred.loc[df_pred['predict'] != df_pred['target']]
df_erros.to_csv(os.path.join(out_path, 'errors.csv'), index=False)

model.save_model(os.path.join(out_path, 'xgb_model'))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:723: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:11:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	valid-merror:0.01183	train-merror:0.01115
[10]	valid-merror:0.00612	train-merror:0.00369
[20]	valid-merror:0.00541	train-merror:0.00174
[30]	valid-merror:0.00541	train-merror:0.00048
[40]	valid-merror:0.00551	train-merror:0.00012
[49]	valid-merror:0.00561	train-merror:0.00008


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:12:09] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from google.colab import files
import gc

# Завантаження файлів
print("Виберіть файл таблиці (.csv або .xlsx) для аналізу")
uploaded_table = files.upload()
table_file = list(uploaded_table.keys())[0]

print("Виберіть файл моделі xgb_model")
uploaded_model = files.upload()
model_file = list(uploaded_model.keys())[0]

# Завантаження таблиці
if table_file.endswith('.csv'):
    df = pd.read_csv(table_file)
elif table_file.endswith(('.xls', '.xlsx')):
    df = pd.read_excel(table_file)
else:
    raise ValueError("Файл таблиці має бути у форматі .csv або .xlsx")

print("Перші 5 рядків таблиці:")
print(df.head())

if 'before' not in df.columns:
    raise ValueError("У таблиці немає колонки 'before'. Переконайтеся, що дані правильні.")

# Налаштування параметрів
max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0

# Перетворення тексту в числовий формат
x_test = []
for x in df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_test.append(x_row)

# Додавання контекстного вікна
def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    pre = [pre for _ in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i : i + pad_size * 2 + 1]:
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data

x_test = np.array(context_window_transform(x_test, pad_size))

# Завантаження моделі
model = xgb.Booster()
model.load_model(model_file)
print("Модель завантажено успішно")

# Виконання передбачення
dtest = xgb.DMatrix(x_test)
predictions = model.predict(dtest)

# Мапінг прогнозів на зрозумілі назви
label_mapping = {
    0: "Category_A",
    1: "Category_B",
    2: "Category_C",
    3: "Category_D",
    # Додайте більше категорій, якщо потрібно
}

try:
    predictions = [label_mapping[int(x)] if int(x) in label_mapping else "Unknown" for x in predictions]
except IndexError:
    raise ValueError("Прогнози моделі містять індекси, які виходять за межі допустимих міток.")

# Додавання результатів до таблиці
df['predictions'] = predictions

print("Результати передбачень:")
print(df[['before', 'predictions']].head())

output_file = "predictions_output.csv"
df.to_csv(output_file, index=False)
print(f"Результати збережено у файл: {output_file}")

files.download(output_file)



Виберіть файл таблиці (.csv або .xlsx) для аналізу


Saving en_test(1)_cleaned.xlsx to en_test(1)_cleaned (3).xlsx
Виберіть файл моделі xgb_model


Saving xgb_model (8) to xgb_model (8) (3)
Перші 5 рядків таблиці:
   sentence_id  token_id      before
0            0         0    Another;
1            0         1  religious;
2            0         2     family;
3            0         3         is;
4            0         4         of;
Модель завантажено успішно
Результати передбачень:
       before predictions
0    Another;  Category_A
1  religious;  Category_A
2     family;  Category_A
3         is;  Category_A
4         of;  Category_A
Результати збережено у файл: predictions_output.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>